Importing Libraries

In [52]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplcursors
import seaborn as sns
import altair as alt
import plotly.express as px

Mounting Dataset

In [53]:
akv16 = pd.read_csv("Laptop Sales dataset.csv")
akv16.head()

,Company,Product,TypeName,Ram,OS,Weight,Price_euros,Screen,ScreenW,ScreenH,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,8,macOS,1.37,1339.69,Standard,2560,1600,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,8,macOS,1.34,898.94,Standard,1440,900,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,8,No OS,1.86,575.00,Full HD,1920,1080,...,No,Intel,2.5,Core i5,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,16,macOS,1.83,2537.45,Standard,2880,1800,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,8,macOS,1.37,1803.60,Standard,2560,1600,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [54]:
akv16.columns

Index(['Company', 'Product', 'TypeName', 'Ram', 'OS', 'Weight', 'Price_euros',
       'Screen', 'ScreenW', 'ScreenH', 'Touchscreen', 'IPSpanel',
       'RetinaDisplay', 'CPU_company', 'CPU_freq', 'CPU_model',
       'PrimaryStorage', 'SecondaryStorage', 'PrimaryStorageType',
       'SecondaryStorageType', 'GPU_company', 'GPU_model'],
      dtype='object')

Data Preprocessing

In [55]:
akv16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1275 non-null   object 
 1   Product               1275 non-null   object 
 2   TypeName              1275 non-null   object 
 3   Ram                   1275 non-null   int64  
 4   OS                    1275 non-null   object 
 5   Weight                1275 non-null   float64
 6   Price_euros           1275 non-null   float64
 7   Screen                1275 non-null   object 
 8   ScreenW               1275 non-null   int64  
 9   ScreenH               1275 non-null   int64  
 10  Touchscreen           1275 non-null   object 
 11  IPSpanel              1275 non-null   object 
 12  RetinaDisplay         1275 non-null   object 
 13  CPU_company           1275 non-null   object 
 14  CPU_freq              1275 non-null   float64
 15  CPU_model            

In [56]:
akv16.drop(['SecondaryStorage','SecondaryStorageType','Product'], axis=1, inplace=True)
akv16.head()

,Company,TypeName,Ram,OS,Weight,Price_euros,Screen,ScreenW,ScreenH,Touchscreen,IPSpanel,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,PrimaryStorageType,GPU_company,GPU_model
0,Apple,Ultrabook,8,macOS,1.37,1339.69,Standard,2560,1600,No,Yes,Yes,Intel,2.3,Core i5,128,SSD,Intel,Iris Plus Graphics 640
1,Apple,Ultrabook,8,macOS,1.34,898.94,Standard,1440,900,No,No,No,Intel,1.8,Core i5,128,Flash Storage,Intel,HD Graphics 6000
2,HP,Notebook,8,No OS,1.86,575.00,Full HD,1920,1080,No,No,No,Intel,2.5,Core i5,256,SSD,Intel,HD Graphics 620
3,Apple,Ultrabook,16,macOS,1.83,2537.45,Standard,2880,1800,No,Yes,Yes,Intel,2.7,Core i7,512,SSD,AMD,Radeon Pro 455
4,Apple,Ultrabook,8,macOS,1.37,1803.60,Standard,2560,1600,No,Yes,Yes,Intel,3.1,Core i5,256,SSD,Intel,Iris Plus Graphics 650


In [57]:
akv16.rename(columns={'Price_euros':'Price','Ram':'RAM'}, inplace=True)

In [58]:
num_columns = len(akv16.columns)
print(num_columns)
akv16.shape

19


(1275, 19)

In [59]:
akv16.isnull().sum()

Company               0
TypeName              0
RAM                   0
OS                    0
Weight                0
Price                 0
Screen                0
ScreenW               0
ScreenH               0
Touchscreen           0
IPSpanel              0
RetinaDisplay         0
CPU_company           0
CPU_freq              0
CPU_model             0
PrimaryStorage        0
PrimaryStorageType    0
GPU_company           0
GPU_model             0
dtype: int64

In [60]:
def remove_outliers_iqr(akv16, column):
    Q1 = akv16[column].quantile(0.25)
    Q3 = akv16[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_cleaned = akv16[(akv16[column] >= lower_bound) & (akv16[column] <= upper_bound)]
    return df_cleaned

akv16 = remove_outliers_iqr(akv16, 'Price')
akv16 = remove_outliers_iqr(akv16, 'Weight')
print(akv16.shape)

(1213, 19)


In [61]:
akv16['CPU_model'] = akv16['CPU_model'].apply(lambda x: 'Core i7' if 'Core i7' in x else x)
akv16['CPU_model'] = akv16['CPU_model'].apply(lambda x: 'Core i5' if 'Core i5' in x else x)
akv16['CPU_model'] = akv16['CPU_model'].apply(lambda x: 'Core i3' if 'Core i3' in x else x)
akv16['CPU_model'] = akv16['CPU_model'].apply(lambda x: 'Celeron Dual Core' if 'Celeron Dual Core' in x else x)


In [62]:
akv16.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1213 entries, 0 to 1274
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Company             1213 non-null   object 
 1   TypeName            1213 non-null   object 
 2   RAM                 1213 non-null   int64  
 3   OS                  1213 non-null   object 
 4   Weight              1213 non-null   float64
 5   Price               1213 non-null   float64
 6   Screen              1213 non-null   object 
 7   ScreenW             1213 non-null   int64  
 8   ScreenH             1213 non-null   int64  
 9   Touchscreen         1213 non-null   object 
 10  IPSpanel            1213 non-null   object 
 11  RetinaDisplay       1213 non-null   object 
 12  CPU_company         1213 non-null   object 
 13  CPU_freq            1213 non-null   float64
 14  CPU_model           1213 non-null   object 
 15  PrimaryStorage      1213 non-null   int64  
 16  PrimaryStor

**Data Visualization**

1. What is the distribution of different laptop brands in the dataset?
2. How does the total price of laptops vary by storage type and company?
3. What is the relationship between laptop price and RAM size for the top laptop companies?
4. How do laptop prices differ across various laptop types (e.g., ultrabook, gaming, notebook)?
5. What is the weight distribution of laptops by brand?
6. What is the market share of different CPU manufacturers (e.g., Intel, AMD) in the dataset?
7. What are the average prices of laptops based on the operating system (e.g., Windows, macOS, Linux)?
8. How do laptop price, weight, and RAM compare across different models?
9. How does the CPU frequency compare between Intel and AMD laptops?

1) Distribution of Laptop Brands


In [63]:

brand_counts = akv16['Company'].value_counts().reset_index()
brand_counts.columns = ['Company', 'Count']

bar_chart_brands = alt.Chart(brand_counts).mark_bar().encode(
    x=alt.X('Company:N', sort='-y', title='Laptop Brands'),  
    y=alt.Y('Count:Q', title='Number of Laptops'),
    color='Company:N', 
    tooltip=['Company:N', 'Count:Q'],
).properties(
    title='Distribution of Laptop Brands',
    width=300,
    height=200,
).interactive()

alt.themes.enable('dark')  

bar_chart_brands.save('1.html')
bar_chart_brands



alt.Chart(...)

2) Total Price by Storage Type and Company

In [64]:
area_data = akv16.groupby(['PrimaryStorageType', 'Company'], as_index=False)['Price'].sum()

company_selection = alt.selection_point(fields=['Company'], bind='legend')

area_plot = alt.Chart(area_data).mark_rect(opacity=0.5).encode(
    x=alt.X('PrimaryStorageType:O', title='Storage Type'),
    y=alt.Y('Price:Q', title='Total Price'),
    color='Company:N',  
    tooltip=['Company:N', 'PrimaryStorageType:O', 'Price:Q']  
    ).add_params(
    company_selection
).transform_filter(
    company_selection
).properties(
    title='Total Price by Storage Type and Company',
    width=300,
    height=200,
).interactive()
alt.themes.enable('dark')  

area_plot.save('2.html')
area_plot

alt.Chart(...)

3) Laptop Price vs RAM Size (Top Companies)

In [65]:
top_companies = akv16['Company'].value_counts().nlargest(5).index
filtered_data = akv16[akv16['Company'].isin(top_companies)]

company_selection = alt.selection_point(fields=['Company'], bind='legend')

scatter_plot = alt.Chart(filtered_data).mark_circle(size=100, opacity=0.7).encode(
    x=alt.X('RAM', title='RAM (GB)'),
    y=alt.Y('Price:Q', title='Price'),
    color=alt.condition(company_selection, 'Company:N', alt.value('lightgray')),  
    tooltip=['Company', 'RAM', 'Price']
).properties(
    title='Laptop Price vs RAM Size (Top Companies)',
    width=550,
    height=250
).add_params(company_selection)

scatter_plot = scatter_plot.interactive()

alt.themes.enable('dark') 
scatter_plot.save('3.html')
scatter_plot


alt.Chart(...)

4) Box Plot: Price Comparison by Laptop Type


In [66]:
histogram_screen_width = alt.Chart(akv16).mark_bar(opacity=0.6).encode(
    x=alt.X('ScreenW:Q', bin=alt.Bin(maxbins=30), title='Screen Width (Pixels)'),
    y=alt.Y('count():Q', title='Count of Laptops', scale=alt.Scale(type='log')), 
    color=alt.Color('count():Q', scale=alt.Scale(scheme='goldred'), title='Count of Laptops'), 
    tooltip=[
        alt.Tooltip('ScreenW:Q', title='Screen Width (Pixels)'),
        alt.Tooltip('count():Q', title='Count of Laptops')  
    ]
).properties(
    title='Distribution of Screen Widths',
    width=500,
    height=250
).interactive()

histogram_screen_width.save('4.html')
histogram_screen_width


alt.Chart(...)

5) Weight Distribution by Laptop Brand

In [67]:
box_plot_weight = alt.Chart(akv16).mark_boxplot().encode(
    x=alt.X('Company:N', title='Brand'),
    y=alt.Y('Weight:Q', title='Weight (Kg)'),
    color='Company'
).properties(
    title='Weight Distribution by Laptop Brand',
    width=550,
    height=230,
).interactive()
alt.themes.enable('dark') 
box_plot_weight.save('5.html')
box_plot_weight


alt.Chart(...)

6) Market Share of CPU Companies

In [68]:
cpu_counts = akv16['CPU_company'].value_counts().reset_index()
cpu_counts.columns = ['CPU_Company', 'Count']
print(cpu_counts)

base = alt.Chart(cpu_counts).encode(
    theta=alt.Theta("Count:Q", stack=True),
    radius=alt.Radius("Count:Q", scale=alt.Scale(type="sqrt", zero=True, rangeMin=150)),
    color=alt.Color("CPU_Company:N", scale=alt.Scale(scheme='dark2'),  legend=None), 
    tooltip=['CPU_Company', 'Count']
).properties(
    title='Market Share of CPU Companies',
    width=500,
    height=210
)

c1 = base.mark_arc(innerRadius=75, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(
    text=alt.Text("Count:Q") 
)

c3 = c1 + c2
c3.save('6.html')
c3.interactive()


  CPU_Company  Count
0       Intel   1152
1         AMD     60
2     Samsung      1


alt.LayerChart(...)

7) Average Laptop Prices by Operating System

In [69]:
laptop_sales_data = akv16[(akv16['OS'] != "No OS")]

average_price_os = laptop_sales_data.groupby('OS')['Price'].mean().reset_index()


bar_chart_os_price = alt.Chart(average_price_os).mark_area(line={'color':'black'},color=alt.Gradient(gradient='linear',
        stops=[alt.GradientStop(color='yellow', offset=0),
               alt.GradientStop(color='red', offset=1)])).encode(
    x=alt.X('OS:N', title='Operating System'),
    y=alt.Y('Price:Q', title='Average Price'),
    tooltip=['OS', 'Price']
).properties(
    title='Average Laptop Prices by Operating System',
    width=400,
    height=190,
)
alt.themes.enable('dark')

bar_chart_os_price.save('2.html')
bar_chart_os_price


alt.Chart(...)

8) Comparing Laptop Price, Weight, and RAM

In [70]:
numerical_cols = ['Price', 'Weight', 'RAM']  

correlation_matrix = akv16[numerical_cols].corr()

fig = px.imshow(correlation_matrix,
                text_auto=True,
                color_continuous_scale='Viridis',
                title='Heatmap of Numerical Features',
                width=500, 
                height=400) 

fig.update_layout(
plot_bgcolor='rgba(0, 0, 0, 0)', 
    paper_bgcolor='rgb(51,51,51)',  
    font_color='white',  
    title_font=dict(size=20, color='white'),
)

fig.write_html('7.html')
fig.show()


9) Comparing CPU frequency of Intel and AMD

In [71]:
import altair as alt
import pandas as pd

intel_data = akv16[akv16['CPU_company'] == 'Intel']
amd_data = akv16[akv16['CPU_company'] == 'AMD']

def create_heatmap(data, title, color_title):
    return alt.Chart(data).mark_rect().encode(
        x=alt.X('CPU_model:N', title='CPU Model', sort='-y'),
        y=alt.Y('CPU_freq:Q', title='CPU Frequency (GHz)'),
        color=alt.Color('CPU_freq:Q', scale=alt.Scale(scheme='blueorange'), title=color_title),
        tooltip=['CPU_model:N', 'CPU_freq:Q', 'count():Q']
    ).properties(
        title=title,
        width=500,
        height=300
    )

heatmap_intel = create_heatmap(intel_data, 'Heatmap of Intel CPU Frequencies', 'CPU Frequency (GHz)')
heatmap_amd = create_heatmap(amd_data, 'Heatmap of AMD CPU Frequencies', 'CPU Frequency (GHz)')

side_by_side_heatmaps = alt.hconcat(heatmap_intel, heatmap_amd).resolve_scale(
    y='shared'  
)

side_by_side_heatmaps.save('8.html')
side_by_side_heatmaps


alt.HConcatChart(...)

In [72]:

fig = px.sunburst(
    akv16,
    path=['CPU_company', 'CPU_model', 'CPU_freq'],  
    values='Price', 
    title="Sunburst Chart of CPU Company, Model, and Frequency",
    color='CPU_company', 
    color_discrete_map={'Intel': 'blue', 'AMD': 'red','Samsung':'yellow'}  
)
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),  
    title_x=0,  
    xaxis=dict(automargin=True),
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgb(51,51,51)', 
    font_color='white',  
    title_font=dict(size=20, color='white'),
)
fig.write_html("sunburst_chart.html")
fig.show()
